In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Call : Agent lifecycle callback

* [Types of callbacks](https://google.github.io/adk-docs/callbacks/types-of-callbacks/)

### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00


In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [3]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

### Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.

In [4]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION


## Session store

In [5]:
import sqlite3
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService, Session

In [32]:
#@title build callback function
from google.adk.agents import LlmAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest

from google.adk.runners import InMemoryRunner # Use InMemoryRunner
from google.genai import types # For types.Content
from typing import Optional

def before_agent_callback(callback_context: CallbackContext) -> Optional[types.Content]:
    agent_name = callback_context.agent_name
    invocation_id = callback_context.invocation_id
    current_state = callback_context.state.to_dict()

    print(f"\n[Before agent Callback] Entering agent: {agent_name} (Inv: {invocation_id})")
    print(f"[Before agent Callback] Current State: {current_state}")

    if current_state is None:
      return types.Content(
          parts=[types.Part(text=f"current_state is None : Agent {agent_name} by before_agent_callback.")],
          role="model"
      )

    else:
      return None

def after_agent_callback(callback_context: CallbackContext) -> Optional[types.Content]:
    agent_name = callback_context.agent_name
    invocation_id = callback_context.invocation_id
    current_state = callback_context.state.to_dict()

    print(f"\n[After agent Callback] Entering agent: {agent_name} (Inv: {invocation_id})")
    print(f"[After agent Callback] Current State: {current_state}")

    if current_state is None:
      return types.Content(
          parts=[types.Part(text=f"current_state is None : Agent {agent_name} by after_agent_callback.")],
          role="model"
      )

    else:
      return None

In [33]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types
from google.adk.tools import load_memory # Tool to query memory
from google.adk.memory import InMemoryMemoryService # Import MemoryService
from google.adk.runners import Runner


PROMPT = """
You are a helpful assistant. Answer user questions using Google Search when needed.
Wirte a simple and clear answer to the user question.
"""

root_agent = LlmAgent(
      name="Callback_agent",
      model=MODEL_NAME,
      instruction=PROMPT,
      description="An assistant that can search the web.",
      tools=[google_search] ,
      after_agent_callback=after_agent_callback,
      before_agent_callback=before_agent_callback
)


## Session

In [34]:
APP_NAME = "Search_Assistant"
USER_ID = "shins"

memory_insert_service = InMemorySessionService()
memory_insert_session = memory_insert_service.create_session(app_name=APP_NAME, user_id=USER_ID)



In [35]:
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=memory_insert_service,
)

In [36]:
query = "What is the exchange rate from US dollars to Korean currency recently ?"

content = types.Content(role='user', parts=[types.Part(text=query)])
events = runner.run(user_id=memory_insert_session.user_id,
                    session_id=memory_insert_session.id,
                    new_message=content)

for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        print(f"Agent Response: {final_response}\n")
    else:
        pprint("No results")



[Before agent Callback] Entering agent: Callback_agent (Inv: e-290ac4ac-146c-4d9d-ba56-e4ea485362e7)
[Before agent Callback] Current State: {}

[After agent Callback] Entering agent: Callback_agent (Inv: e-290ac4ac-146c-4d9d-ba56-e4ea485362e7)Agent Response: Based on recent search results, the exchange rate from US dollars to South Korean Won (KRW) is approximately:

*   **1 US Dollar (USD) is equal to about 1421 to 1434 South Korean Won (KRW).**

Please note that exchange rates fluctuate constantly based on market conditions. For the most up-to-the-minute rate, it's always best to check a reliable currency converter or financial service.


[After agent Callback] Current State: {}


## End of notebook